<a href="https://colab.research.google.com/github/DarshanPatel0919/Deep-Learning/blob/master/LabAssignment2_201701436.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Darshan Patel 

201701436

Deep Learning

Lab 2

In [ ]:
import tensorflow as tf
import numpy as np
import pylab as plt

In [ ]:
##### Returns an activation function based on index 
activation_function = [tf.math.sigmoid,tf.nn.softmax,tf.nn.relu,tf.nn.tanh]

#### Basic Forward Propogation for given input with given parameters    
def NN(input,layers,activation_functions,mean,sigma,start,end):
  ### A will be input for next layer
  A = input[start:end]
  prev_layer_size = tf.cast(tf.shape(input)[1],dtype=tf.int64)

  ### Generalized layer array has ith element as number of nodes in ith layer
  for layer in range(len(layers)):
    current_layer_size = layers[layer]
    ### Formulas of forward propogation
    W = tf.random.normal([prev_layer_size,current_layer_size],mean,sigma,dtype=tf.dtypes.float64)
    B = tf.repeat(tf.random.normal([1,current_layer_size],mean,sigma,dtype=tf.dtypes.float64),repeats= end - start,axis=0)
    Z = tf.add(tf.linalg.matmul(A,W),B)
    A = tf.Variable(activation_function[layer](Z),dtype=tf.dtypes.float64)
    prev_layer_size = current_layer_size
  return A

### Batch Forward propogation
def NN_batch(input,layers,activation_functions,mean,sigma,batches):
  all_batch_predictions = []
  m = tf.shape(input)[0]
  for batch_size in batches:
    start = 0
    single_batch_prediction = []
    while start<m:
      pre_data = NN(input,layers,activation_functions,mean,sigma,start,min(start+batch_size,m))
      for i in range(tf.shape(pre_data)[0]):
        rest_data = []
        for j in range(10):
          rest_data += [pre_data[i][j]]
        single_batch_prediction += [rest_data]
      start += batch_size
    all_batch_predictions += [single_batch_prediction]
  return all_batch_predictions

### Different Nodes in given layer
def NN_node(input,layerss,activation_functions,mean,sigma):
  return [NN(input,layers,activation_functions,mean,sigma,0,tf.shape(input)[0]) for layers in layerss]

### Different Activation functions for given layer
def NN_activation(input,layers,activation_functionss,mean,sigma):
  return [NN(input,layers,activation_functions,mean,sigma,0,tf.shape(input)[0]) for activation_functions in activation_functionss]

# **Digit Classification**

In [ ]:
### Loading the dataset
data = tf.keras.datasets.mnist.load_data(path="mnist.npz")[0]

### x and y are respectively  images and  labels 
x , y = data
x = tf.reshape(x,[tf.shape(x)[0],tf.shape(x)[1]*tf.shape(x)[2]]) #Input will be #Examples(m) * #Features(n)

x = tf.cast(x,dtype=tf.float64)
y = tf.cast(y,dtype=tf.float64)

In [ ]:
### In layers first n-1 elements are for #nodes in hidden layers
### Last element of layers will be the #nodes in output layer 
layers = tf.constant([512,256,128,10],dtype=tf.int64)

### activation_functions stores activation function id of respective layer
activation_functions = [0,2,1,1] 

### mean and sigma for generating the random weights
mean = tf.constant(0,dtype=tf.dtypes.float64)
sigma = tf.constant(1,dtype=tf.dtypes.float64)

m = tf.cast(tf.shape(x)[0],dtype=tf.int64)

###Sample Output
output_forward_prop = NN(x,layers,activation_functions,mean,sigma,0,m)

y_d=[]
for i in range(tf.shape(x)[0]):
  y_d.append(tf.argmax(output_forward_prop[i,:]))

conf_mat = tf.math.confusion_matrix(y,y_d)
accuracy = tf.cast(tf.linalg.trace(conf_mat),dtype=tf.int64)/m #Accuracy = fraction of results which were predicted correctly

print("Confusion Matrix")
tf.print(conf_mat)
print("Accuracy :",end=" ")
tf.print(accuracy)

Varying the Batch Sizes

In [ ]:
### batches of different sizes
batches=[50,100,500,1000,5000,10000]

output_forward_prop_set = NN_batch(x,layers,activation_functions,mean,sigma,batches)

accuracy = []
for output_forward_prop in output_forward_prop_set:
  print(np.shape(output_forward_prop))
  y_d=[tf.argmax(output_forward_prop[i]) for i in range(tf.shape(x)[0])]
  conf_mat = tf.math.confusion_matrix(y,y_d)
  accuracy += [tf.cast(tf.linalg.trace(conf_mat),dtype=tf.int64)/m]

print("Batches vs Accuracy")
plt.plot(batches,accuracy)

Observation : Here we can see that changing the size of batches does not affect (changes are random) the accuracy because the data is untrained

******************************************************************************************************************************

Varying Number of Nodes in Layers

In [ ]:
### nodes of different numbers
layerss=[[512,256,128,10],[5,5,5,10],[128,256,512,10],[2,4,8,10],[55,25,55,10]]

output_forward_prop_set = NN_node(x,layerss,activation_functions,mean,sigma)

accuracy = []
for output_forward_prop in output_forward_prop_set:
  y_d=[tf.argmax(output_forward_prop[i,:]) for i in range(tf.shape(x)[0])]
  conf_mat = tf.math.confusion_matrix(y,y_d)
  accuracy += [tf.cast(tf.linalg.trace(conf_mat),dtype=tf.int64)/m]


print("Different Nodes sets vs Accuracy")
plt.plot([0,1,2,3,4],accuracy)

Observation : Here we can see that changing the number of nodes does not affect the accuracy because the data is untrained

******************************************************************************************************************************

Varying Activation Functions in Layers

In [ ]:
activation_functionss=[[0,1,2,3],[0,0,0,0],[2,1,2,0],[1,0,3,2],[0,2,1,1]]

output_forward_prop_set = NN_activation(x,layers,activation_functionss,mean,sigma)

accuracy = []
for output_forward_prop in output_forward_prop_set:
  y_d=[tf.argmax(output_forward_prop[i,:]) for i in range(tf.shape(x)[0])]
  conf_mat = tf.math.confusion_matrix(y,y_d)
  accuracy += [tf.cast(tf.linalg.trace(conf_mat),dtype=tf.int64)/m]

print("Different Activation function sets vs Accuracy")
plt.plot([0,1,2,3,4],accuracy)

Observation : Here we can see that changing the activity functions does not affect the accuracy because the data is untrained

#Boston Housing Problem

In [ ]:
### Loading the dataset
data = tf.keras.datasets.boston_housing.load_data(path="boston_housing.npz", test_split=0.2, seed=113)[0]

### x and y are respectively  images and  labels 
x , y = data

x = tf.cast(x,dtype=tf.float64)
y = tf.cast(y,dtype=tf.float64)
x.shape

Varying Number of Nodes in Layers

In [ ]:
### nodes of different numbers
layerss=[[512,256,128,1],[5,5,5,1],[128,256,512,1],[2,4,8,1],[55,25,55,1]]
layers = layerss[0]

output_forward_prop_set = NN_node(x,layerss,activation_functions,mean,sigma)

MSE = []
for y_d in output_forward_prop_set:
  MSE += [tf.keras.losses.MSE(y,tf.transpose(y_d))]
  
print("Different Nodes sets vs MSE")
plt.plot([0,1,2,3,4],MSE)

Observation : Here we can see that changing the number of nodes does not affect the MSE because the data is untrained

******************************************************************************************************************************

Varying Activation Functions in Layers

In [ ]:
activation_functionss=[[0,0,0,0],[1,1,1,1],[2,2,2,2],[3,3,3,3],[0,2,1,1]]

output_forward_prop_set = NN_activation(x,layers,activation_functionss,mean,sigma)

MSE = []
for y_d in output_forward_prop_set:
  MSE += [tf.keras.losses.MSE(y,tf.transpose(y_d))]
  
print("Different Nodes sets vs MSE")
plt.plot([0,1,2,3,4],MSE)

Observation : Here we can see that changing the activity functions does not actually affect, the effects are random the MSE because the data is untrained